# 2019 OK

## Assumptions/ accountability-sourced documentation:

* Values for chosen orbit found here: https://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2019%20OK
* Location: L11, Gemini South
* Time: 2010/01/01 to 2020/01/01, midnights

## Current questions:

* Do I need to cite things in-code?
* Vertical or horizontal tables? See below
* How do I add units to the line that currently says 'object' (see cell 2)
* Do we need a maximum difference function or class? What do I need to know about OOP for class things?

In [11]:
%matplotlib inline

import numpy as np
import astropy as ap
import astropy.units as u
from astropy.table import QTable
import matplotlib as plt
# import astroquery as aq - fix anaconda and virenv before getting this

In [12]:
# Values table thing goes here

# vertical version

Element1 = ["e", "a", "q", "i", "node", "peri", "M", "t_p", "period", "n", "Q"]
Value1 = [.7571011351666629, 1.866845215104908, .4534545835685292, 2.089875053674596,
         302.0443761140395, 106.0972546563977, 101.8116827265041, 2458737.014932156260, 
         931.6673871166022, .3864039945781041, 3.280235846641287]
Unit1 = ["Null", "AU", "AU", "deg", "deg", "deg", "deg", "unsure", "days", "deg/d", "AU"]

hi = QTable([Element1, Value1, Unit1],
          names=('Element', 'Value', 'Units'),
          meta={'name': '2019OK data'})

# horizontal version

Element2 = ["e", "a", "q", "i", "node", "peri", "M", "t_p", "period", "n", "Q"]
Value2 = [.7571011351666629, 1.866845215104908, .4534545835685292, 2.089875053674596,
         302.0443761140395, 106.0972546563977, 101.8116827265041, 2458737.014932156260, 
         931.6673871166022, .3864039945781041, 3.280235846641287]
Value2 = np.array(Value)
Value2 = np.transpose(Value)
Unit2 = [u.dimensionless_unscaled, u.AU, u.AU, u.deg, u.deg, u.deg, u.deg, u.dimensionless_unscaled, 
         u.d, u.deg/u.d, u.AU]
bye = QTable(Value2*Unit2, names = Element2,)
print(hi)
bye

Element       Value        Units 
------- ------------------ ------
      e 0.7571011351666629   Null
      a  1.866845215104908     AU
      q 0.4534545835685292     AU
      i  2.089875053674596    deg
   node  302.0443761140395    deg
   peri  106.0972546563977    deg
      M  101.8116827265041    deg
    t_p  2458737.014932156 unsure
 period  931.6673871166022   days
      n 0.3864039945781041  deg/d
      Q  3.280235846641287     AU


e,a,q,i,node,peri,M,t_p,period,n,Q
object,object,object,object,object,object,object,object,object,object,object
0.7571011351666629,1.866845215104908 AU,0.4534545835685292 AU,2.089875053674596 deg,302.0443761140395 deg,106.0972546563977 deg,101.8116827265041 deg,2458737.014932156,931.6673871166022 d,0.3864039945781041 deg / d,3.280235846641287 AU


In [13]:
# JPL integration
# Values = .elements() something here?

In [14]:
# OpenOrb integration

In [16]:
# difference function
# function or class?

def max_diff_metric(var1, var2):
    
    # solution for the 360deg problem
    
    return RA_diff, DEC_diff